Dependencies

In [49]:
import subprocess
import json
import re
import os
import cma
import csv
import pandas as pd
import numpy as np
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
import pymoo
print(pymoo.__version__)
import matplotlib.pyplot as plt
from pymoo.algorithms.moo.moead import MOEAD
from pymoo.util.ref_dirs import get_reference_directions

0.6.1.3


Hiperparameters values

In [50]:
# Valores discretos para las capas y las neuronas
if True:    
    layers_range = [1, 2, 3, 4]
    neurons_options = [2, 4, 8, 16, 32, 64, 128]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.0001, 0.7)
    mu_range = (0.0001, 1.0)
else:
    layers_range = [1]
    neurons_options = [2, 4]
    offline_options = [0, 1]
    error_function_options = [0, 1]

    # Rango continuo para eta y mu
    eta_range = (0.01, 0.1)
    mu_range = (0.01, 0.1)

In [64]:
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\nTest error \(Mean \+- SD\):\s+([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)\s*Test\s+error \(Mean \+- SD\): ([\d\.eE+-]+) \+- ([\d\.eE+-]+)"
#train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"
train_test_ccr_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Correctly\s+Classified\s+Ratio\s+\(CCR\):\s*([\d\.eE+-]+)\s+\((\d+)/(\d+)\)"

CMA-ES ALGORTIHM OPTIMIZATION - LOOKING FOR THE BEST SET OF PARAMETERS

In [69]:
def run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function):
    # Construir el comando con las nuevas flags
    if train_file != "train_nomnist.dat":
        iterations = "1000"
    else:
        iterations = "500"
    command = [
        "./bin/la2", "-t", train_file, "-T", test_file,
        "-e", str(eta), "-m", str(mu), "-l", str(layers), "-h", str(neurons), "-i", iterations,
        "-f", str(error_function)
    ]
    # Añadir el modo offline si corresponde
    if offline == 1:
        command.append("-o")

    # Ejecutar el programa
    result = subprocess.run(command, capture_output=True, text=True)

    # Buscar los valores de error
    match = re.search(train_test_ccr_regex, result.stdout, re.DOTALL)
    if match:
        train_mean, train_std, test_mean, test_std, ccr, correct, total = match.groups()
        return {
            "train_mean": float(train_mean),
            "train_std": float(train_std),
            "test_mean": float(test_mean),
            "test_std": float(test_std),
            "ccr": float(ccr),
            "correct": int(correct),
            "total": int(total)
        }
    return None

In [53]:
def evaluate_fitness(params, train_file, test_file):
    # Limitar eta y mu dentro de sus rangos permitidos
    eta = max(min(params[0], eta_range[1]), eta_range[0])
    mu = max(min(params[1], mu_range[1]), mu_range[0])

    # Limitar otros parámetros como antes
    layers = int(max(min(params[2], max(layers_range)), min(layers_range)))
    neurons_index = int(max(min(params[3], len(neurons_options) - 1), 0))
    neurons = neurons_options[neurons_index]
    offline = int(max(min(params[4], 1), 0))
    error_function = int(max(min(params[5], 1), 0))

    # Ejecutar el programa y obtener resultados
    result = run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function)

    if result:
        return result["test_mean"], result  # Devuelve el test_mean y el resto del resultado
    return float("inf"), None  # Penalizar si hay fallos y no hay resultado


In [54]:
def save_to_csv(filename, results):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Encabezado del CSV
        writer.writerow(["Eta", "Mu", "Layers", "Neurons", "Offline", "Error Function",
                         "Train Mean", "Train Std", "Test Mean", "Test Std"])
        # Escribir cada fila de resultados
        for row in results:
            writer.writerow(row)

In [55]:
# Bucle principal actualizado
def optimize_with_cma_es(dataset_key):
    if dataset_key not in datasets:
        print(f"Dataset '{dataset_key}' no encontrado.")
        return

    # Archivos de entrenamiento y prueba
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]

    # Configuración inicial de CMA-ES
    initial_params = [0.01, 0.5, 2, 3, 1, 0]  # [eta, mu, layers, neurons_index, offline, error_function]

    sigma = 0.1 # Rango de búsqueda inicial
    es = cma.CMAEvolutionStrategy(
    initial_params,
    sigma,
    {
        'bounds': [
            [eta_range[0], mu_range[0], 1, 0, 0, 0],  # Límite inferior
            [eta_range[1], mu_range[1], max(layers_range), len(neurons_options) - 1, 1, 1]  # Límite superior
        ]
    }
)

    # Resultados almacenados
    results = []

    # Bucle de optimización
    while not es.stop():
        solutions = es.ask()
        fitness = []
        for solution in solutions:
            # Evaluar cada solución
            test_mean, result = evaluate_fitness(solution, train_file, test_file)
            fitness.append(test_mean)  # Usar solo test_mean como fitness

            if result:  # Guardar resultados solo si son válidos
                eta, mu = solution[0], solution[1]
                layers = int(max(min(solution[2], max(layers_range)), min(layers_range)))
                neurons_index = int(max(min(solution[3], len(neurons_options) - 1), 0))
                neurons = neurons_options[neurons_index]
                offline = int(max(min(solution[4], 1), 0))
                error_function = int(max(min(solution[5], 1), 0))

                # Almacenar resultados
                results.append([
                    eta, mu, layers, neurons, offline, error_function,
                    result["train_mean"], result["train_std"], result["test_mean"], result["test_std"]
                ])

        # Informar a CMA-ES
        es.tell(solutions, fitness)
        print(f"Generación {es.result.iterations}: Mejor error de prueba = {min(fitness)}")

    return results

MULTI-OBJETIVE OPTIMIZATION: FOR THE CHEAPEST SET OF PARAMETERS

In [56]:
if False:
    import numpy as np
    import random
    import subprocess
    import re
    from concurrent.futures import ThreadPoolExecutor

    # Definir parámetros de optimización
    LAYER_OPTIONS = [1, 2, 3, 4]  # Valores discretos
    NEURON_OPTIONS = [2, 4, 8, 16, 32, 64, 128]  # Valores discretos
    ETA_RANGE = (0.0001, 0.9)  # Continuo
    MU_RANGE = (0.0001, 0.9)  # Continuo
    POP_SIZE = 25
    GENERATIONS = 40
    # Expresión regular para capturar errores de entrenamiento y prueba
    train_test_regex = r"Train\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)\s*Test\s+error\s+\(Mean\s+\+-\s+SD\):\s*([\d\.eE+-]+)\s+\+-\s+([\d\.eE+-]+)"


    # Función para generar un individuo
    def generate_individual():
        layers = random.choice(LAYER_OPTIONS)
        neurons = random.choice(NEURON_OPTIONS)
        eta = random.uniform(*ETA_RANGE)
        mu = random.uniform(*MU_RANGE)
        offline = random.choice([0, 1])  # Discreto
        error_function = random.choice([0, 1])  # Discreto
        return [layers, neurons, eta, mu, offline, error_function]

    # Función para inicializar la población
    def initialize_population(size):
        return [generate_individual() for _ in range(size)]

    # Evaluar la población en paralelo
    def evaluate_population(population, dataset_key):
        dataset_files = datasets[dataset_key]
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(lambda ind: evaluate(ind, dataset_files), population))
        return results

    # Función de evaluación
    def evaluate(individual, dataset_files):
        train_file = dataset_files["train"]
        test_file = dataset_files["test"]

        if train_file != "./dat/train_nomnist.dat":
            iterations = "1000"
        else:
            iterations = "500"
        
        layers, neurons, eta, mu, offline, error_function = individual

        # Construir el comando para ejecutar la red neuronal
        command = [
            "./bin/la2",
            "-l", str(layers),
            "-h", str(neurons),
            "-e", str(eta),
            "-m", str(mu),
            "-o", str(offline),
            "-f", str(error_function),
            "-t", train_file,
            "-T", test_file,
            "-i", iterations
        ]

        # Ejecutar el comando y capturar la salida
        result = subprocess.run(command, capture_output=True, text=True)

        # Extraer los errores usando la regex
        match = re.search(train_test_regex, result.stdout)
        if match:
            train_error = float(match.group(1))
            test_error = float(match.group(3))
        else:
            # Penalizar si no se puede ejecutar correctamente
            train_error = float("inf")
            test_error = float("inf")

        # Calcular el número total de neuronas
        total_neurons = layers * neurons

        # Retornar las funciones objetivo
        return train_error, test_error, total_neurons

    # Función para calcular si un individuo domina a otro
    def dominates(ind1, ind2):
        return all(x <= y for x, y in zip(ind1, ind2)) and any(x < y for x, y in zip(ind1, ind2))

    # Función para actualizar el frente de Pareto
    def update_pareto_front(front, individual, objectives):
        to_remove = []
        dominated = False

        for i, (ind, obj) in enumerate(front):
            if dominates(objectives, obj):
                to_remove.append(i)
            elif dominates(obj, objectives):
                dominated = True

        for i in sorted(to_remove, reverse=True):
            del front[i]

        if not dominated:
            front.append((individual, objectives))

    # Selección por torneo binario
    def tournament_selection(population, objectives):
        i, j = random.sample(range(len(population)), 2)
        if dominates(objectives[i], objectives[j]):
            return population[i]
        elif dominates(objectives[j], objectives[i]):
            return population[j]
        return random.choice([population[i], population[j]])

    # Cruce uniforme
    def crossover(parent1, parent2):
        return [
            (p1 + p2) / 2 if isinstance(p1, float) else random.choice([p1, p2])
            for p1, p2 in zip(parent1, parent2)
        ]

    # Mutación
    def mutate(individual, mutation_rate=0.2):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                if i in [0, 1]:  # Discretos
                    individual[i] = random.choice(LAYER_OPTIONS if i == 0 else NEURON_OPTIONS)
                elif i in [2, 3]:  # Continuos
                    range_val = ETA_RANGE if i == 2 else MU_RANGE
                    individual[i] = random.uniform(*range_val)
                elif i in [4, 5]:  # Binarios
                    individual[i] = 1 - individual[i]

    # Algoritmo evolutivo con parada anticipada
    def evolutionary_algorithm(dataset_key):
        population = initialize_population(POP_SIZE)
        pareto_front = []
        max_no_improvement_gens = 10  # Número de generaciones sin mejora
        no_improvement_counter = 0  # Contador de generaciones sin mejora
        best_test_error = float('inf')  # Mejor error de prueba registrado

        for gen in range(GENERATIONS):
            new_population = []
            objectives = evaluate_population(population, dataset_key)

            # Actualizar el frente de Pareto
            for ind, obj in zip(population, objectives):
                update_pareto_front(pareto_front, ind, obj)

            # Obtener el mejor test_error actual
            current_best_test_error = min(obj[1] for _, obj in pareto_front)  # Segundo objetivo: test_error

            # Verificar si ha habido mejora
            if current_best_test_error < best_test_error:
                best_test_error = current_best_test_error
                no_improvement_counter = 0  # Reiniciar contador
            else:
                no_improvement_counter += 1

            # Parada anticipada si no hay mejora significativa
            if no_improvement_counter >= max_no_improvement_gens:
                print(f"Parada anticipada en generación {gen + 1}. No hubo mejora significativa en las últimas {max_no_improvement_gens} generaciones.")
                break

            # Crear nueva población
            for _ in range(POP_SIZE):
                parent1 = tournament_selection(population, objectives)
                parent2 = tournament_selection(population, objectives)
                child = crossover(parent1, parent2)
                mutate(child)
                new_population.append(child)

            population = new_population
            print(f"Generación {gen + 1}: Frente de Pareto tiene {len(pareto_front)} soluciones")

        return pareto_front


GRID SEARCH 

In [60]:
import itertools
import os
import csv

def grid_search(datasets, eta_values, mu_values, layers_values, neurons_values, offline_values, error_function_values, output_dir="resultados_final"):
    """
    Performs a grid search over hyperparameters for the specified datasets.

    Parameters:
        datasets (dict): A dictionary with dataset keys and their train/test file paths.
        eta_values (list): Learning rates to test.
        mu_values (list): Momentum values to test.
        layers_values (list): Number of layers to test.
        neurons_values (list): Number of neurons per layer to test.
        offline_values (list): Whether to use offline training [0, 1].
        error_function_values (list): Error functions to test [0, 1].
        output_dir (str): Directory to save the results.

    Returns:
        None: Results are saved in CSV files.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    # Iterate over each dataset
    for dataset_key, files in datasets.items():
        train_file = files["train"]
        test_file = files["test"]

        # Open a CSV file to save results for this dataset
        output_file = os.path.join(output_dir, f"grid_search_{dataset_key}.csv")

        # Check if the file already exists to append results
        file_exists = os.path.isfile(output_file)

        with open(output_file, mode="a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            # Write the header if the file is new
            if not file_exists:
                writer.writerow(["eta", "mu", "layers", "neurons", "offline", "error_function",
                                 "train_mean", "train_std", "test_mean", "test_std", "ccr", "correct", "total"])

            # Create all combinations of hyperparameters
            combinations = itertools.product(eta_values, mu_values, layers_values,
                                              neurons_values, offline_values, error_function_values)

            
            # Run each combination
            for eta, mu, layers, neurons, offline, error_function in combinations:
                print(f"Running {dataset_key} with eta={eta}, mu={mu}, layers={layers}, neurons={neurons}, offline={offline}, error_function={error_function}")
                result = run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function)
                if result:
                    writer.writerow([eta, mu, layers, neurons, offline, error_function,
                                     result["train_mean"], result["train_std"],
                                     result["test_mean"], result["test_std"], result["ccr"], result["correct"], result["total"]])

    print(f"Grid search completed. Results saved in {output_dir}")

def grid_search_combinations(datasets, combinations, output_dir="resultados3"):
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    # Iterate over each dataset
    for dataset_key, files in datasets.items():
        train_file = files["train"]
        test_file = files["test"]

        # Open a CSV file to save results for this dataset
        output_file = os.path.join(output_dir, f"grid_search_{dataset_key}.csv")

        # Check if the file already exists to append results
        file_exists = os.path.isfile(output_file)

        with open(output_file, mode="a", newline="") as csvfile:
            writer = csv.writer(csvfile)
            # Write the header if the file is new
            if not file_exists:
                writer.writerow(["eta", "mu", "layers", "neurons", "offline", "error_function",
                                 "train_mean", "train_std", "test_mean", "test_std", "ccr", "correct", "total"])

            # Run each combination
            for combination in combinations:
                eta, mu, layers, neurons, offline, error_function = combination
                print(f"Running {dataset_key} with eta={eta}, mu={mu}, layers={layers}, neurons={neurons}, offline={offline}, error_function={error_function}")
                result = run_program(train_file, test_file, eta, mu, layers, neurons, offline, error_function)
                if result:
                    writer.writerow([eta, mu, layers, neurons, offline, error_function,
                                     result["train_mean"], result["train_std"],
                                     result["test_mean"], result["test_std"], result["ccr"], result["correct"], result["total"]])

    print(f"Grid search completed. Results saved in {output_dir}")

Grid search

In [61]:
# Define hyperparameter ranges
eta_values = [0.1, 0.5, 0.9]
mu_values = [0.1, 0.5, 0.9]
layers_values = [2, 3, 4, 5]
neurons_values = [4, 8, 16, 24, 32]
offline_values = [0, 1]
error_function_values = [0, 1]


grid_search({"xor": 
                {
                "train": "./dat/train_xor.dat",
                "test": "./dat/test_xor.dat"
                }
            }, 
            eta_values, mu_values, layers_values, neurons_values, offline_values, [0], "resultados_final_mse")
grid_search({"nomnist": 
                {
                "train": "./dat/train_nomnist.dat",
                "test": "./dat/test_nomnist.dat"
                }
            }, 
            eta_values, mu_values, layers_values, neurons_values, offline_values, [0], "resultados_final_mse")
grid_search({"nomnist": 
                {
                "train": "./dat/train_nomnist.dat",
                "test": "./dat/test_nomnist.dat"
                }
            }, 
            eta_values, mu_values, layers_values, neurons_values, offline_values, [1], "resultados_final_ce")
# # Run the grid search
grid_search({"compas": 
                {
                "train": "./dat/train_compas.dat",
                "test": "./dat/test_compas.dat"
                }
            }, 
            eta_values, mu_values, layers_values, neurons_values, offline_values, [0], "resultados_final_mse")






Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=4, offline=0, error_function=0
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=4, offline=0, error_function=1
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=4, offline=1, error_function=0
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=4, offline=1, error_function=1
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=8, offline=0, error_function=0
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=8, offline=0, error_function=1
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=8, offline=1, error_function=0
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=8, offline=1, error_function=1
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=16, offline=0, error_function=0
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=16, offline=0, error_function=1
Running nomnist with eta=0.1, mu=0.5, layers=2, neurons=16, offline=1, error_function=0
Running nomnist with eta=0.1, mu=0.5, la

KeyboardInterrupt: 

Datasets

In [ ]:
datasets = {
    
    "compas": {
        "train": "./dat/train_compas.dat",
        "test": "./dat/test_compas.dat"
    },
    "nomnist": {
        "train": "./dat/train_nomnist.dat",
        "test": "./dat/test_nomnist.dat"
    }
}

PARETO FOR THE CMA RESULTS:

In [89]:
import csv

def load_cma_results_and_get_pareto(csv_file):
    """
    Carga los resultados de CMA-ES desde un archivo CSV, calcula el conjunto de soluciones
    no dominadas (frente de Pareto) en base a error de entrenamiento, error de test,
    y número de neuronas, e imprime los no dominados.
    
    :param csv_file: Ruta del archivo CSV con los resultados de CMA-ES.
    :return: Una tupla (no_dominados, dominados), donde ambos son listas de individuos completos.
    """
    # Cargar resultados desde el archivo CSV
    results = []
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Leer encabezados
        for row in reader:
            results.append([float(x) if i < 6 else float(x) for i, x in enumerate(row)])
    
    # Extraer objetivos: train_error, test_error, total_neurons
    objectives = [
        (result[6], result[8], result[2] * result[3])  # Train error, Test error, Total neurons
        for result in results
    ]

    # Identificar no dominados y dominados
    no_dominados = []
    dominados = []

    for i, obj_i in enumerate(objectives):
        is_dominated = False
        for j, obj_j in enumerate(objectives):
            if i != j and dominates(obj_j, obj_i):  # Si obj_j domina a obj_i
                is_dominated = True
                break
        if is_dominated:
            dominados.append(results[i])
        else:
            no_dominados.append(results[i])

    # Imprimir no dominados
    print("No Dominados:")
    for nd in no_dominados:
        print(nd)

    return no_dominados, dominados


def dominates(ind1, ind2):
    """
    Determina si el individuo ind1 domina al individuo ind2.
    Un individuo domina a otro si es igual o mejor en todos los objetivos
    y estrictamente mejor en al menos uno.
    :param ind1: Primer individuo (objetivos).
    :param ind2: Segundo individuo (objetivos).
    :return: True si ind1 domina a ind2, False en caso contrario.
    """
    return all(x <= y for x, y in zip(ind1, ind2)) and any(x < y for x, y in zip(ind1, ind2))

In [90]:
import matplotlib.pyplot as plt
import os

def obtain_best_combination(pareto_results, dataset_key):
    """
    Analiza las combinaciones no dominadas y encuentra la mejor combinación basada en
    la relación beneficio-esfuerzo. Genera y guarda un gráfico para visualizar los resultados.
    
    :param pareto_results: Lista de resultados no dominados.
    :param dataset_key: Nombre del dataset (usado para nombrar el archivo de la imagen).
    :return: Índice de la mejor combinación y su métrica.
    """
    # Crear directorio si no existe
    output_dir = "resultados/imagenes"
    os.makedirs(output_dir, exist_ok=True)
    
    # Ordenar por complejidad
    pareto_results = sorted(pareto_results, key=lambda x: x[2] * x[3])  # Ordenar por total_neurons

    best_index = None
    best_efficiency = -float("inf")
    efficiencies = []

    # Graficar relación beneficio-esfuerzo
    plt.figure(figsize=(10, 6))
    for i in range(1, len(pareto_results)):
        prev = pareto_results[i - 1]
        curr = pareto_results[i]

        # Calcular incrementos
        prev_neurons = prev[2] * prev[3]
        curr_neurons = curr[2] * curr[3]
        complexity_increase = curr_neurons - prev_neurons

        train_gain = prev[6] - curr[6]  # Mejora en error de entrenamiento
        test_gain = prev[8] - curr[8]  # Mejora en error de prueba

        # Relación beneficio-esfuerzo
        total_gain = train_gain + test_gain
        if complexity_increase > 0:
            efficiency = total_gain / complexity_increase
        else:
            efficiency = 0  # Evitar divisiones por cero

        efficiencies.append(efficiency)

        # Actualizar la mejor combinación
        if efficiency > best_efficiency:
            best_efficiency = efficiency
            best_index = i

        # Añadir al gráfico
        plt.scatter(complexity_increase, total_gain, label=f"Comb {i}", s=80)

    # Graficar
    plt.xlabel("Complexity Growth (Total Neurons)")
    plt.ylabel("Total Gain (Train + Test Error)")
    plt.title(f"Benefit-Effort Ratio for Non-Dominated Combinations - {dataset_key}")
    plt.axhline(0, color='gray', linestyle='--')
    plt.legend()
    plt.grid(True)
    
    # Guardar la imagen
    output_file = os.path.join(output_dir, f"non_dominated_analysis_{dataset_key}.png")
    plt.savefig(output_file)
    plt.close()

    print(f"Gráfico guardado en: {output_file}")

    # Retornar la mejor combinación
    return best_index, pareto_results[best_index], best_efficiency


In [91]:
import re
from collections import defaultdict
import os

def leer_vectores(filepath):
    """
    Lee un archivo de salida y organiza los resultados por semilla.
    
    :param filepath: Ruta al archivo de salida.
    :return: Diccionario que contiene los datos organizados por semilla.
    """
    # Patrones para extraer la semilla y los datos de iteración/errores
    seed_pattern = re.compile(r"SEED (\d+)")
    iteration_pattern = re.compile(r"Iteration (\d+)\s+Training error:\s+([\d.]+)\s+Test error:\s+([\d.]+)")
    
    # Diccionario para almacenar resultados
    results = defaultdict(lambda: {"iterations": [], "training_errors": [], "test_errors": []})
    
    # Variables auxiliares
    current_seed = None

    # Leer el archivo línea por línea
    with open(filepath, "r") as file:
        for line in file:
            # Identificar la semilla actual
            seed_match = seed_pattern.search(line)
            if seed_match:
                current_seed = int(seed_match.group(1))
                continue
            
            # Extraer los datos de iteración/errores si hay una semilla activa
            if current_seed is not None:
                iteration_match = iteration_pattern.search(line)
                if iteration_match:
                    iteration = int(iteration_match.group(1))
                    training_error = float(iteration_match.group(2))
                    test_error = float(iteration_match.group(3))
                    
                    # Agregar los datos al diccionario de la semilla actual
                    results[current_seed]["iterations"].append(iteration)
                    results[current_seed]["training_errors"].append(training_error)
                    results[current_seed]["test_errors"].append(test_error)
    
    # Convertir el diccionario en un formato estándar
    return dict(results)

def execute_best_combinations(dataset_key, combination, output_filename):
    """
    Ejecuta una combinación de parámetros y guarda la salida en un archivo de texto.
    
    :param dataset_key: Clave del dataset.
    :param combination: Lista con los parámetros a ejecutar.
    :param output_filename: Nombre del archivo de salida.
    :return: Salida de la ejecución como texto.
    """
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]
    layers, neurons, eta, mu, offline, error_function = combination[:6]

    command = [
        "./bin/la2",
        "-l", str(layers),
        "-h", str(neurons),
        "-e", str(eta),
        "-m", str(mu),
        "-o", str(offline),
        "-f", str(error_function),
        "-t", train_file,
        "-T", test_file,
        "-i", "1000"
    ]

    # Ejecutar el comando
    result = subprocess.run(command, capture_output=True, text=True)

    # Guardar la salida en un archivo
    output_dir = "resultados/outputs"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, output_filename)
    with open(output_path, "w") as file:
        file.write(result.stdout)

    return result.stdout



def create_convergence_plot(data, title, output_filename, max_iterations=1000):
    """
    Crea un gráfico de convergencia basado en los datos de la semilla número 5.
    
    :param data: Diccionario con los datos organizados por semillas.
    :param title: Título del gráfico.
    :param output_filename: Nombre del archivo donde se guardará el gráfico.
    :param max_iterations: Número máximo de iteraciones a considerar.
    """
    # Verificar si la semilla número 5 está presente
    if 5 not in data:
        print("La semilla número 5 no está disponible en los datos.")
        return
    
    # Extraer los datos de la semilla número 5
    seed_5_data = data[5]
    iterations = seed_5_data["iterations"]
    training_errors = seed_5_data["training_errors"]
    test_errors = seed_5_data["test_errors"]
    
    # Validar si el penúltimo valor no corresponde a la última iteración menos uno
    if len(iterations) > 1 and iterations[-2] != iterations[-1] - 1:
        print("Advertencia: Las iteraciones no son consecutivas al final. Se usarán datos hasta el penúltimo valor.")
        iterations = iterations[:-1]
        training_errors = training_errors[:-1]
        test_errors = test_errors[:-1]
    
    # Crear el gráfico
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, training_errors, label="Training Error", color="orange")
    plt.plot(iterations, test_errors, label="Test Error", color="blue")
    plt.xlabel("Iterations")
    plt.ylabel("Error")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    
    # Guardar el gráfico
    plt.savefig(output_filename)
    plt.close()
    print(f"Gráfico de convergencia guardado en: {output_filename}")


def run_full_study():
    """
    Ejecuta el estudio completo según los pasos solicitados.
    """
    report_lines = []

    for dataset_key in datasets.keys():
        # 1. Cargar resultados de CMA
        csv_file = f"resultados/resultados_cma_{dataset_key}.csv"
        with open(csv_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            headers = next(reader)
            results = [list(map(float, row)) for row in reader]

        # 2. Obtener el mejor resultado (menor train y test error)
        best_result = min(results, key=lambda x: (x[6], x[8]))

        # 3. Obtener soluciones no dominadas y la mejor combinación
        non_dominated, _ = load_cma_results_and_get_pareto(csv_file)
        _, best_pareto, _ = obtain_best_combination(non_dominated, dataset_key)

        # 4. Ejecutar el mejor resultado y el mejor no dominado
        print(f"Ejecutando la mejor combinación posible para {dataset_key}...")
        best_output = execute_best_combinations(dataset_key, best_result, f"mejor_posible_{dataset_key}_output.txt")
        print(f"Ejecutando la mejor combinación no dominada para {dataset_key}...")
        pareto_output = execute_best_combinations(dataset_key, best_pareto, f"mejor_no_dominado_{dataset_key}_output.txt")

        # 5. Obtener vectores y crear gráficos de convergencia
        print("Generando gráficos de convergencia...")
        best_data = leer_vectores(f"resultados/outputs/mejor_posible_{dataset_key}_output.txt")
        pareto_data = leer_vectores(f"resultados/outputs/mejor_no_dominado_{dataset_key}_output.txt")

        create_convergence_plot(
            best_data, "Error Convergence - Best Result", f"resultados/imagenes/error_convergence_best_{dataset_key}.png"
        )
        create_convergence_plot(
            pareto_data, "Error Convergence - Optimal Pareto", f"resultados/imagenes/error_convergence_optimal_{dataset_key}.png"
        )

        # 6. Agregar al informe
        report_lines.append(f"Dataset: {dataset_key}")
        report_lines.append(f"Mejor resultado posible: {best_result}")
        report_lines.append(f"Mejor combinación no dominada: {best_pareto}")
        report_lines.append("")

    # Guardar el informe
    with open("resultados/outputs/informe.txt", "w") as report_file:
        report_file.write("\n".join(report_lines))
    print("Informe generado en: resultados/outputs/informe.txt")


# Ejecutar el estudio completo
run_full_study()

No Dominados:
[0.06838442144772591, 0.5776018154095421, 1.0, 8.0, 0.0, 0.0, 0.0010454, 0.0001972, 0.0010454, 0.0001972]
[0.6868002794449244, 0.9561334134732801, 3.0, 8.0, 0.0, 0.0, 9.40767e-05, 2.49431e-05, 9.40767e-05, 2.49431e-05]
[0.6302985944180677, 0.9899932136577357, 2.0, 8.0, 0.0, 0.0, 0.000137165, 4.7651e-05, 0.000137165, 4.7651e-05]
[0.6999868729841289, 0.6699145473880292, 3.0, 16.0, 0.0, 0.0, 6.30208e-05, 1.43253e-05, 6.30208e-05, 1.43253e-05]
Gráfico guardado en: resultados/imagenes/non_dominated_analysis_xor.png
Ejecutando la mejor combinación posible para xor...
Ejecutando la mejor combinación no dominada para xor...
Generando gráficos de convergencia...
Gráfico de convergencia guardado en: resultados/imagenes/error_convergence_best_xor.png
Gráfico de convergencia guardado en: resultados/imagenes/error_convergence_optimal_xor.png
No Dominados:
[0.0011908753113457767, 0.6013920420459544, 1.0, 8.0, 0.0, 0.0, 0.513153, 0.0264822, 0.514273, 0.0295396]
[0.0012166491673840903, 0